In [5]:
from math import exp

In [7]:
def dot(v,w):
    return sum(v_i*w_i for v_i,w_i in zip(v,w))

def sigmoid(t):
    return 1 / (1 + exp(-t))

In [18]:
# nonsmoothed step function
def step_function(x):
    return 1 if x >= 0 else 0

def perceptron_output(weights,bias,x):
    calc = dot(weights,x) + bias
    return step_function(calc)
#---------------

def neuron_output(weights,inputs):
    return sigmoid(dot(weights,inputs))

def feed_forward(neural_network,input_vector):
    outputs = []
    
    # process one layer at a time
    for layer in neural_network:
        input_with_bias = input_vector + [1]
        output = [neuron_output(neuron, input_with_bias) for neuron in layer]
        outputs.append(output)
        
        input_vector = output
        
    return outputs

In [19]:
xor_network = [[[20,20,-30],[20,20,-10]],[[-60,60,-30]]]

for x in [0,1]:
    for y in [0,1]:
        print(x,y,feed_forward(xor_network,[x,y])[-1])

0 0 [9.38314668300676e-14]
0 1 [0.9999999999999059]
1 0 [0.9999999999999059]
1 1 [9.383146683006828e-14]


In [20]:
['a','b'] + [1]

['a', 'b', 1]

In [ ]:
# Backpropagation
# the function of this is to place errors, when working through a model, to infer errors for hidden layers
def backpropagate(network, input_vector,targets):
    hidden_outputs, outputs = feed_forward(network,input_vector)
    
    # output * (1-output) is from derivative of sigmoid, see logistic regression
    output_deltas = [output * (1-output) * ( 1- target) 
                    for output,target in zip(outputs,targets)]
    
    # adjust weights for output_layer, one neuron at a time
    for i, output_neuron in enumerate(network[-1]):
        # focus on the ith output layer neuron, remember the bias of 1!
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            # adjust the jth weight based on both
            # this neuron's delta and it's jth input
            output_neuron[j] -= output_deltas[i] * hidden_output
            
    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1-hidden_output) *
                    dot(output_deltas, [n[i] for n in network[-1]])
                    for i, hidden_output in enumerate(hidden_outputs)]
    
    # adjust weights for hidden layer, one neuron at a time
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input